In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from glob import glob
import gc
import psutil
from ase.io import read
import ase
import os

%matplotlib inline
print(os.listdir("../inputs"))
print('ase version', ase.__version__)

def show_ram_usage():
    py = psutil.Process(os.getpid())
    print('RAM usage: {} GB'.format(py.memory_info()[0]/2. ** 30))
show_ram_usage()

['test.csv', 'train.csv', 'potential_energy.csv', 'magnetic_shielding_tensors.csv', 'scalar_coupling_contributions.csv', 'sample_submission.csv', 'mulliken_charges.csv', 'structures.csv', 'dipole_moments.csv', 'structures']
ase version 3.17.0
RAM usage: 0.12180328369140625 GB


In [2]:
train = pd.read_csv('../inputs/train.csv', index_col='id')
test = pd.read_csv('../inputs/test.csv', index_col='id')

print(display(train.head(), test.head()))
# First we want to extract mol_name, [atom_index0, atom_index1] as a list
def extract_base(base):
    mol_name = base['molecule_name'].to_numpy()
    atom_index0 = base['atom_index_0'].to_numpy()
    atom_index1 = base['atom_index_1'].to_numpy()
    return mol_name, atom_index0, atom_index1

mol_train, atom_index0_train, atom_index1_train = extract_base(train)
mol_test, atom_index0_test, atom_index1_test = extract_base(test)

#-----------------------------------------------------------------
print(f'There are {train.shape[0]} rows in train data.')
print(f'There are {test.shape[0]} rows in test data.')
print('')
print(f'There are {mol_train.shape[0]} rows in mol_train data.')
print(f'There are {mol_test.shape[0]} rows in mol_test data.')
print('')
print(f"There are {train['molecule_name'].nunique()} distinct molecules in train data.")
print(f"There are {test['molecule_name'].nunique()} distinct molecules in test data.")
print(f"There are {train['atom_index_0'].nunique()} unique atoms.")
print(f"There are {train['type'].nunique()} unique types.")
print('')
print(f'There are {atom_index0_train.shape[0]} rows in atom_index0_train data.')
print(f'There are {atom_index0_test.shape[0]} rows in atom_index0_test data.')
print('')
print(f'There are {atom_index1_train.shape[0]} rows in atom_index1_train data.')
print(f'There are {atom_index1_test.shape[0]} rows in atom_index1_test data.')
print('')
# ---------------------------------------------------------------
show_ram_usage()

/home/sher/anaconda2/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,dsgdb9nsd_000001,2,0,1JHC,84.8074


,molecule_name,atom_index_0,atom_index_1,type
id,,,,
4658147,dsgdb9nsd_000004,2,0,2JHC
4658148,dsgdb9nsd_000004,2,1,1JHC
4658149,dsgdb9nsd_000004,2,3,3JHH
4658150,dsgdb9nsd_000004,3,0,1JHC
4658151,dsgdb9nsd_000004,3,1,2JHC


None
There are 4658147 rows in train data.
There are 2505542 rows in test data.

There are 4658147 rows in mol_train data.
There are 2505542 rows in mol_test data.

There are 85003 distinct molecules in train data.
There are 45772 distinct molecules in test data.
There are 29 unique atoms.
There are 8 unique types.

There are 4658147 rows in atom_index0_train data.
There are 2505542 rows in atom_index0_test data.

There are 4658147 rows in atom_index1_train data.
There are 2505542 rows in atom_index1_test data.

RAM usage: 0.5275154113769531 GB


In [3]:
# We will define our functions in this panel

# Let's define the cut-off function
def fcut(Ri, Rc):
    if Ri <= Rc:
        y = 0.5*(np.cos(np.pi*Ri/Rc)+1.0)
    else:
        y = 0
    return y

# Convert  to mol object using ase
def convert_xyz_to_mol(mol_id):
    mol = read(f'../inputs/structures/{mol_id}.xyz')
    natom = len(mol)
    atoms = mol.get_atomic_numbers()
    return mol, natom, atoms

# We can calculate the distance r_b with atom indices
# As well as acquire r_i, theta_i for the molecule
def get_rb_ri_theta(mol, natom, atoms, atom_index0, atom_index1):
    r_b = mol.get_distance(atom_index0, atom_index1)
    r_i = []
    angles = []
    theta_i = []
    for i in range((len(mol))):
        if i == atom_index0:
            theta_i.append(0)
            r_i.append(0)
            continue
        if i == atom_index1:
            theta_i.append(0)
            r_i.append(0)
            continue
        angle1 = mol.get_angle(atom_index0, i, atom_index1)
        angle2 = mol.get_angle(atom_index0, atom_index1, i)
        angle3 = mol.get_angle(atom_index1, atom_index0, i)
        angles.append([angle1, angle2, angle3])
        # Get theta from center of bond
        theta = 180 - angle3 - (angle1/2.0)
        theta_i.append(theta)
        # Get distance from center of bond
        r = (r_b/2.0)/(np.sin(np.radians(theta))*np.sin(np.radians(angle1/2.0)))
        r_i.append(r) 
    return r_b, r_i, theta_i


# NOTE: r_b is a single value whereas r_i and theta_i will equal to natom

# Let's Define the radial function G1_atomic
def get_G1_atomic(Ri, eta, mu):
     return np.exp(-eta*(Ri-mu)**2) * fcut(Ri, Rc)


def get_G1(mol_id, index_0, index_1):
    mol, natom, atoms = convert_xyz_to_mol(mol_id)
    r_b, r_i, theta_i = get_rb_ri_theta(mol, natom, atoms, index_0, index_1)
    G1_par = np.zeros(shape=(npar, 5)) # H C N O F 
    for i in range(natom):
        if i == index_0:
            continue
        if i == index_1:
            continue
        for j in range(npar):
            eta_m = p1[j][0]
            mu_m =  p1[j][1]
            G1_temp =  get_G1_atomic(r_i[i], eta_m, mu_m)
            if atoms[i] == 1:
                G1_par[j][0] = G1_par[j][0] + G1_temp
            if atoms[i] == 6:
                G1_par[j][1] = G1_par[j][1] + G1_temp
            if atoms[i] == 7:
                G1_par[j][2] = G1_par[j][2] + G1_temp
            if atoms[i] == 8:
                G1_par[j][3] = G1_par[j][3] + G1_temp
            if atoms[i] == 9:
                G1_par[j][4] = G1_par[j][4] + G1_temp
    return G1_par, r_b

In [5]:
# Globally defined parameters:
# for simplicity we will define eta, mu with few variables
# Also worth experimenting with various cut-off radius Rc, here marked as 6.0

Rc = 6.0
eta = [1.00, 0.5, 0.1 ,0.01]
mu = [0.0, 2.0, 4.0, 6.0]


p1 =  [[i,j] for i in eta for j in mu]
npar = len(eta)*len(mu)

r_b_train = []
G1_train = []

for i, j in enumerate(mol_train):
    G1, r_b = get_G1(j, atom_index0_train[i], atom_index1_train[i])
    G1_train.append(G1)
    r_b_train.append(r_b)
print(len(r_b_train))
print(len(G1_train))
print(len(G1_train[0]))
print(len(G1_train[1]))
# ---------------------------------------------------------------
show_ram_usage()

/home/sher/anaconda2/lib/python3.7/site-packages/ipykernel/__main__.py:42: RuntimeWarning: divide by zero encountered in double_scalars


4658147
4658147
16
16
RAM usage: 4.700996398925781 GB


In [9]:
np.save('rb_train.npy', r_b_train)
np.save('G1_train.npy', G1_train)
print('Saved!')

Saved!


In [11]:
# First let's do a sanity check
print('First 10 values of r_b : ')
print(r_b_train[0:10])
print('')
print('First 5 values of G1')
print(G1_train[0:5])

First 10 values of r_b : 
[1.091953059411922, 1.7831197560438021, 1.7831474962399783, 1.7831566855725496, 1.0919516188628118, 1.7831576600993326, 1.783148394630673, 1.0919463791331034, 1.7831478722270893, 1.091947541273265]

First 5 values of G1
[array([[1.25068683e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.93171413e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00087843e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.73965845e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.60776149e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.99810713e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [4.54818374e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.89617893e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
  

In [ ]:
# To flatten the G1 function so that there are 4658147 rows and 80 (16*5) columns

G1_array = np.zeros(shape=(len(G1_train), 80))
for i, row1 in enumerate(G1_train):
    G1_array[i] = row1.flatten()        
# ---------------------------------------------------------------
show_ram_usage()
